In [1]:
import numpy as np

In [2]:
from bokeh.io import push_notebook, output_notebook, show
from bokeh.layouts import row
from bokeh.plotting import figure, ColumnDataSource
from bokeh.models import PanTool, BoxZoomTool, HoverTool, CrosshairTool, ResetTool

In [3]:
from sklearn import datasets
from sklearn.decomposition import PCA

In [4]:
from PIL import Image
import base64
from io import BytesIO

In [5]:
def make_scatter2d_images(x, y, names=None, image_files=None, clustering=None):
    source_data = dict(x=x, y=y)
    if names is not None:
        source_data["desc"] = names
        tooltips_desc = """<span style="font-size: 17px; font-weight: bold;">@desc</span>"""
    else:
        tooltips_desc = ""
    if image_files is not None:
        source_data["imgs"] = image_files
        tooltips_images = """
            <div>
                <img
                    src="@imgs" height="42" alt="@imgs" width="42"
                    style="float: left; margin: 0px 15px 15px 0px;"
                    border="2"
                ></img>
            </div>
        """
    else:
        tooltips_images = ""
    if clustering is not None:
        color_map = make_color_map(clustering)
        cluster_colors = [color_map[c] for c in clustering]
        source_data['cluster_color'] = cluster_colors
    source = ColumnDataSource(data=source_data)
    hover = HoverTool(tooltips="""
        <div>
            {}
            <div>
                {}
                <span style="font-size: 15px; color: #966;">[$index]</span>
            </div>
            <div>
                <span style="font-size: 15px;">Location</span>
                <span style="font-size: 10px; color: #696;">($x, $y)</span>
            </div>
        </div>
        """.format(tooltips_images, tooltips_desc))
    p = figure(width=500, height=500)
    for t in [PanTool(), BoxZoomTool(), hover, CrosshairTool(), ResetTool()]:
        p.add_tools(t)
    if clustering is not None:
        p.circle(x='x', y='y',
                 fill_color='cluster_color',
                 line_color='cluster_color',
                 size=5, source=source)
    else:
        p.circle(x='x', y='y', size=5, source=source)
    return p


In [6]:
def gnp2im(image_np, bit_depth_scale_factor):
    """Converts an image stored as a 2-D grayscale Numpy array into a PIL image."""
    return Image.fromarray((image_np * bit_depth_scale_factor).astype(np.uint8), mode='L')

In [7]:
def to_base64(png):
    return "data:image/png;base64," + base64.b64encode(png).decode("utf-8")

In [8]:
def get_thumbnails(data, bit_depth_scale_factor):
    thumbnails = []
    for gnp in data:
        im = gnp2im(gnp, bit_depth_scale_factor)
        memout = BytesIO()
        im.save(memout, format='png')
        thumbnails.append(to_base64(memout.getvalue()))
    return thumbnails

In [9]:
output_notebook()

Loading BokehJS ...

In [10]:
# n = 32
# h = 32
# w = 32

In [11]:
dataset = datasets.load_digits()

In [12]:
data = dataset['images']
labels = dataset['target']

In [13]:
n, h, w = data.shape

In [14]:
#data = np.random.rand(n, h, w)

In [15]:
pca = PCA(n_components=2)

In [16]:
z = pca.fit_transform(data.reshape((n, h * w)))

In [17]:
z.shape

(1797, 2)

In [18]:
x = z[:, 0]
y = z[:, 1]

In [19]:
bit_depth_scale_factor = 255

In [20]:
thumbnails = get_thumbnails(data, bit_depth_scale_factor)

In [21]:
thumbnails

['data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAAgAAAAICAAAAADhZOFXAAAAPElEQVR4nAXBQQqAMAxFwZefgFLw/vfsptWQxhmD3HEbgn16JqLOM6IQ7cJBdECDsKLLEKpcXBh8rw/4ASPFFQCK90yVAAAAAElFTkSuQmCC',
 'data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAAgAAAAICAAAAADhZOFXAAAAMklEQVR4nDXGMQ6AQAgAsIJedPL/D73FBIKTnRrYcy+J6CLpcpDePU7S9TAk6D9IYik+EyAMCdI96LAAAAAASUVORK5CYII=',
 'data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAAgAAAAICAAAAADhZOFXAAAAOUlEQVR4nAXBhw2AIAAAsLIkUf+/1AFB1DbgOdMuMg8vkZHqS6bGu5B9fVjJWrcFgnlNZSlZMtuIfo6LEv1ImO+4AAAAAElFTkSuQmCC',
 'data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAAgAAAAICAAAAADhZOFXAAAAOklEQVR4nAXBQQ5AMBAAwNlNHQT//yhRUl0zwdNzD6m/6/wIM04HKWtmEajLFg1R9yPpY4zSiNuy+gEWghXpQuAv3wAAAABJRU5ErkJggg==',
 'data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAAgAAAAICAAAAADhZOFXAAAAN0lEQVR4nCXGQQ6AIAwAsG4i//+rGuKM4MGeCutCorY/q3JOjZp9hPSM3Z1SRB12TfOePSRidT6lsxHZCRAoQgAAAABJRU5ErkJggg==',
 'data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAAgAAAAICAAAAADhZOFXAAAAN0lEQVR4nAXBWw5AQAwAwGkV8eP+5xTxC

In [22]:
colors = ["#%02x%02x%02x" % (int(r), int(g), 150) for r, g in zip(50+2*x, 30+2*y)]

In [23]:
figure_width = 400
figure_height = 400

In [26]:

""" Demonstration of how to register event callbacks using an adaptation
of the color_scatter example from the bokeh gallery
"""
import numpy as np

from bokeh import events
from bokeh.io import output_file, show
from bokeh.layouts import column, row
from bokeh.models import Button, CustomJS, Div
from bokeh.plotting import figure


def display_event(div, x, y, thumbnails, figure_width, figure_height, attributes=[], style = 'float:left;clear:left;font_size=13px'):
    "Build a suitable CustomJS to display the current event in the div model."
    return CustomJS(args=dict(div=div, x=x, y=y, thumbnails=thumbnails, figure_width=figure_width, figure_height=figure_height), code="""
        var attrs = %s; var args = []; var n = x.length;
        
        var test_x;
        var test_y;
        for (var i = 0; i < attrs.length; i++) {
            if (attrs[i] == 'x') {
                test_x = Number(cb_obj[attrs[i]]);
            }
            
            if (attrs[i] == 'y') {
                test_y = Number(cb_obj[attrs[i]]);
            }
        }
    
        var minDiffIndex = -1;
        var minDiff = 99999;
        var squareDiff;
        for (var i = 0; i < n; i++) {
            squareDiff = (test_x - x[i]) ** 2 + (test_y - y[i]) ** 2;
            if (squareDiff < minDiff) {
                minDiff = squareDiff;
                minDiffIndex = i;
            }
        }
        
        var img_tag_attrs = "height='" + (figure_height * 0.5) + "' width='" + (figure_width * 0.5) + "' style='float: left; margin: 0px 15px 15px 0px;' border='2'";
        var img_tag = "<div><img src='" + thumbnails[minDiffIndex] + "' " + img_tag_attrs + "></img></div>";
        var line = "<span style=%r>Index: " + minDiffIndex + "</span>" + img_tag + "\\n";
        div.text = "";
        var text = div.text.concat(line);
        var lines = text.split("\\n")
        if (lines.length > 35)
            lines.shift();
        div.text = lines.join("\\n");
    """ % (attributes, style))

p = figure(plot_width=figure_width, plot_height=figure_height)
p.scatter(x, y, fill_color=colors, fill_alpha=0.6, line_color=None)

div = Div(width=400, height=p.plot_height, height_policy="fixed")
layout = row(p, div)

point_attributes = ['x', 'y', 'sx', 'sy']

p.js_on_event(events.MouseMove, display_event(div, x, y, thumbnails, figure_width, figure_height, attributes=point_attributes))
#p.js_on_event(events.Tap, display_event(div, x, y, thumbnails, figure_width, figure_height, attributes=point_attributes))

show(layout)

In [27]:
p = figure(width=figure_width, height=figure_height)
p.scatter(x, y, fill_color=colors, fill_alpha=0.6, line_color=None)

GlyphRenderer(id='1178', ...)

In [9]:
# """ Demonstration of how to register event callbacks using an adaptation
# of the color_scatter example from the bokeh gallery
# """
# import numpy as np

# from bokeh import events
# from bokeh.io import output_file, show
# from bokeh.layouts import column, row
# from bokeh.models import Button, CustomJS, Div
# from bokeh.plotting import figure

# def display_event(div, x, y, thumbnails, figure_width, figure_height, attributes=[], style = 'float:left;clear:left;font_size=13px'):
#     "Build a suitable CustomJS to display the current event in the div model."
#     return CustomJS(args=dict(div=div, x=x, y=y, thumbnails=thumbnails, figure_width=figure_width, figure_height=figure_height), code="""
#         var attrs = %s; var args = []; var n = x.length;
        
#         var test_x;
#         var test_y;
#         for (var i = 0; i < attrs.length; i++) {
#             if (attrs[i] == 'x') {
#                 test_x = Number(cb_obj[attrs[i]]);
#             }
            
#             if (attrs[i] == 'y') {
#                 test_y = Number(cb_obj[attrs[i]]);
#             }
#         }
    
#         var minDiffIndex = -1;
#         var minDiff = 99999;
#         var squareDiff;
#         for (var i = 0; i < n; i++) {
#             squareDiff = (test_x - x[i]) ** 2 + (test_y - y[i]) ** 2;
#             if (squareDiff < minDiff) {
#                 minDiff = squareDiff;
#                 minDiffIndex = i;
#             }
#         }
        
#         var img_tag_attrs = "height='" + figure_height + "' width='" + figure_width + "' style='float: left; margin: 0px 15px 15px 0px;' border='2'";
#         var img_tag = "<div><img src='" + thumbnails[minDiffIndex] + "' " + img_tag_attrs + "></img></div>";
#         var line = "<span style=%r>Index: " + minDiffIndex + "</span>" + img_tag + "\\n";
#         div.text = "";
#         var text = div.text.concat(line);
#         var lines = text.split("\\n")
#         if (lines.length > 35)
#             lines.shift();
#         div.text = lines.join("\\n");
#     """ % (attributes, style))

In [28]:
div = Div(width=400, height=p.plot_height, height_policy="fixed")
layout = row(p, div)

In [31]:
point_attributes = ['x', 'y', 'sx', 'sy']
p_scatter.js_on_event(events.Tap, display_event(div, x, y, thumbnails, figure_width, figure_height, attributes=point_attributes))

In [36]:
x.shape

(32,)

In [37]:
y.shape

(32,)

In [41]:
""" Demonstration of how to register event callbacks using an adaptation
of the color_scatter example from the bokeh gallery
"""
import numpy as np

from bokeh import events
from bokeh.io import output_file, show
from bokeh.layouts import column, row
from bokeh.models import Button, CustomJS, Div
from bokeh.plotting import figure


def display_event(div, x, y, thumbnails, figure_width, figure_height, attributes=[], style = 'float:left;clear:left;font_size=13px'):
    "Build a suitable CustomJS to display the current event in the div model."
    return CustomJS(args=dict(div=div, x=x, y=y, thumbnails=thumbnails, figure_width=figure_width, figure_height=figure_height), code="""
        var attrs = %s; var args = []; var n = x.length;
        
        var test_x;
        var test_y;
        for (var i = 0; i < attrs.length; i++) {
            if (attrs[i] == 'x') {
                test_x = Number(cb_obj[attrs[i]]);
            }
            
            if (attrs[i] == 'y') {
                test_y = Number(cb_obj[attrs[i]]);
            }
        }
    
        var minDiffIndex = -1;
        var minDiff = 99999;
        var squareDiff;
        for (var i = 0; i < n; i++) {
            squareDiff = (test_x - x[i]) ** 2 + (test_y - y[i]) ** 2;
            if (squareDiff < minDiff) {
                minDiff = squareDiff;
                minDiffIndex = i;
            }
        }
        
        var img_tag_attrs = "height='" + figure_height + "' width='" + figure_width + "' style='float: left; margin: 0px 15px 15px 0px;' border='2'";
        var img_tag = "<div><img src='" + thumbnails[minDiffIndex] + "' " + img_tag_attrs + "></img></div>";
        var line = "<span style=%r>Index: " + minDiffIndex + "</span>" + img_tag + "\\n";
        div.text = "";
        var text = div.text.concat(line);
        var lines = text.split("\\n")
        if (lines.length > 35)
            lines.shift();
        div.text = lines.join("\\n");
    """ % (attributes, style))

def display_event(div, x, y, thumbnails, figure_width, figure_height, attributes=[], style = 'float:left;clear:left;font_size=13px'):
    "Build a suitable CustomJS to display the current event in the div model."
    return CustomJS(args=dict(div=div, x=x, y=y, thumbnails=thumbnails, figure_width=figure_width, figure_height=figure_height), code="""
        var attrs = %s; var args = []; var n = x.length;
        var line = "<span style=%r>Hello world.</span>\\n";
        div.text = "hello world.";
    """ % (attributes, style))

colors = ["#%02x%02x%02x" % (int(r), int(g), 150) for r, g in zip(50+2*x, 30+2*y)]

figure_width = 500
figure_height = 500
p = figure(width=figure_width, height=figure_height)
p.scatter(x, y, fill_color=colors, fill_alpha=0.6, line_color=None)

div = Div(width=400, height=p.plot_height, height_policy="fixed")

layout = row(p_scatter, div)
p.js_on_event(events.Tap, display_event(div, x, y, thumbnails, figure_width, figure_height, attributes=point_attributes))
show(layout)


In [34]:
show(layout)

In [47]:

""" Demonstration of how to register event callbacks using an adaptation
of the color_scatter example from the bokeh gallery
"""
import numpy as np

from bokeh import events
from bokeh.io import output_file, show
from bokeh.layouts import column, row
from bokeh.models import Button, CustomJS, Div
from bokeh.plotting import figure


def display_event(div, x, y, thumbnails, figure_width, figure_height, attributes=[], style = 'float:left;clear:left;font_size=13px'):
    "Build a suitable CustomJS to display the current event in the div model."
    return CustomJS(args=dict(div=div, x=x, y=y, thumbnails=thumbnails, figure_width=figure_width, figure_height=figure_height), code="""
        var attrs = %s; var args = []; var n = x.length;
        
        var test_x;
        var test_y;
        for (var i = 0; i < attrs.length; i++) {
            if (attrs[i] == 'x') {
                test_x = Number(cb_obj[attrs[i]]);
            }
            
            if (attrs[i] == 'y') {
                test_y = Number(cb_obj[attrs[i]]);
            }
        }
    
        var minDiffIndex = -1;
        var minDiff = 99999;
        var squareDiff;
        for (var i = 0; i < n; i++) {
            squareDiff = (test_x - x[i]) ** 2 + (test_y - y[i]) ** 2;
            if (squareDiff < minDiff) {
                minDiff = squareDiff;
                minDiffIndex = i;
            }
        }
        
        var img_tag_attrs = "height='" + figure_height + "' width='" + figure_width + "' style='float: left; margin: 0px 15px 15px 0px;' border='2'";
        var img_tag = "<div><img src='" + thumbnails[minDiffIndex] + "' " + img_tag_attrs + "></img></div>";
        var line = "<span style=%r>Index: " + minDiffIndex + "</span>" + img_tag + "\\n";
        div.text = "";
        var text = div.text.concat(line);
        var lines = text.split("\\n")
        if (lines.length > 35)
            lines.shift();
        div.text = lines.join("\\n");
    """ % (attributes, style))

# x = np.random.random(size=100) * 100
# y = np.random.random(size=100) * 100
colors = ["#%02x%02x%02x" % (int(r), int(g), 150) for r, g in zip(50+2*x, 30+2*y)]
#radii = np.random.random(size=4000) * 1.5


figure_width = 500
figure_height = 500
p = figure(plot_width=figure_width, plot_height=figure_height)
p.scatter(x, y, fill_color=colors, fill_alpha=0.6, line_color=None)

div = Div(width=400, height=p.plot_height, height_policy="fixed")

layout = row(p, div)
p.js_on_event(events.Tap, display_event(div, x, y, thumbnails, figure_width, figure_height, attributes=point_attributes))
show(layout)

In [24]:
p_scatter = make_scatter2d_images(x, y, names=None, image_files=thumbnails, clustering=None)

In [18]:
import bokeh.palettes
grayscale_palette = bokeh.palettes.gray(256)

In [21]:
def make_image_plot(img):
    # Define the bounding coordinates to generate the Mandelbrot image in. You
    # can play around with these.
    
    # <32
    min_x = -2.0
    max_x = 2.0
    
    # 32
    min_y = -1.0
    max_y = 1.0

    # Use the functions above to create a scalar image (2D array of numbers)
    #img = np.zeros((1024, 1536), dtype = np.uint8)
    #create_fractal(min_x, max_x, min_y, max_y, img, 20)

    # create a figure, setting the x and y ranges to the appropriate data bounds
    p = figure(plot_width=int(900/2), plot_height=int(900/2),
                x_range = [min_x, max_x], y_range = [min_y, max_y])

    # Fill in the missing parameters to use the `image` renderer to
    # display the Mandelbrot image color mapped with the palette 'Spectral11'
    #
    # NOTE: the `image` renderer can display many images at once, so it takes
    # **lists** of images, coordinates, and palettes. Remember to supply sequences
    # for these parameters, even if you are just supplying one.
    p.image(image=[img],             # image data
             x=[min_x],               # lower left x coord
             y=[min_y],               # lower left y coord
             dw=[max_x-min_x],        # *data space* width of image
             dh=[max_y-min_y],        # *data space* height of image
             palette=grayscale_palette,    # palette name
    )

    return p

p_image = make_image_plot(data[0, :, :])

In [25]:
show(row(p_scatter, p_image), notebook_handle=True)

In [45]:
div = Div(width=400, height=p.plot_height, height_policy="fixed")

In [49]:
x.shape

(4000,)

In [58]:
x.tolist()

[72.44949992519105,
 83.44268747852458,
 25.74018525879074,
 41.54605669877912,
 51.44966525459621,
 44.533533892767906,
 1.626283883742352,
 10.481628912369011,
 94.78727717204862,
 8.440406968032043,
 27.869678540092814,
 27.16699180051053,
 4.593130269288171,
 60.665013150451095,
 68.62999471817814,
 64.44990377337578,
 2.635729794302555,
 96.90347717746015,
 4.556040747997658,
 55.615991292888154,
 97.98311316339623,
 30.982717741212696,
 65.83423111115,
 47.09372004194591,
 87.05283933802613,
 0.9625227492923938,
 83.09323973672925,
 46.09151122435258,
 62.376887492515685,
 24.452503434331696,
 73.56040130854772,
 69.46966026161363,
 61.42724130974987,
 4.11867331599729,
 35.35896156868965,
 93.68565481729162,
 96.66457263156603,
 75.96969596524966,
 28.90861160045247,
 32.022712314458246,
 7.705395880470977,
 47.0931663329526,
 58.22920117927299,
 83.77468963069869,
 79.77215503239796,
 82.75870495217028,
 11.994059177647964,
 32.61545864869685,
 5.040933537258063,
 27.4505895614

In [ ]:
#button = Button(label="Button", button_type="success")
#layout = column(button, row(p, div))

# ## Events with no attributes
# button.js_on_event(events.ButtonClick, display_event(div)) # Button click
# p.js_on_event(events.LODStart, display_event(div))         # Start of LOD display
# p.js_on_event(events.LODEnd, display_event(div))           # End of LOD display

# ## Events with attributes
# point_attributes = ['x', 'y', 'sx', 'sy']                  # Point events
# wheel_attributes = point_attributes + ['delta']            # Mouse wheel event
# pan_attributes = point_attributes + ['delta_x', 'delta_y'] # Pan event
# pinch_attributes = point_attributes + ['scale']            # Pinch event

# point_events = [
#     events.Tap, events.DoubleTap, events.Press, events.PressUp,
#     events.MouseMove, events.MouseEnter, events.MouseLeave,
#     events.PanStart, events.PanEnd, events.PinchStart, events.PinchEnd,
# ]

# for event in point_events:
#     p.js_on_event(event, display_event(div, attributes=point_attributes))

# p.js_on_event(events.MouseWheel, display_event(div, attributes=wheel_attributes))
# p.js_on_event(events.Pan,        display_event(div, attributes=pan_attributes))
# p.js_on_event(events.Pinch,      display_event(div, attributes=pinch_attributes))

# #output_file("js_events.html", title="JS Events Example")